In [3]:
# ! pip install pandas 
# ! pip install numpy
# ! pip install gensim.models

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim import models
from gensim.models import KeyedVectors
import ast
import re
import pickle

# 코사인 유사도
import numpy as np
from numpy import dot
from numpy.linalg import norm

# 유사 태그 출력
from heapq import nlargest
from heapq import nsmallest

# # 모델에 없는 단어들 형태소 분석 
# from konlpy.tag import Okt
# from konlpy.utils import pprint
# okt = Okt()

#### DATA

In [2]:
# 데이터 받아오기 
chart_tag = pd.read_pickle("C:/Users/SAMSUNG/Desktop/KUBIG/2024 Fall/추천시스템/Mine/DATA/chart.pickle")
genre_tag = pd.read_pickle("C:/Users/SAMSUNG/Desktop/KUBIG/2024 Fall/추천시스템/Mine/DATA/genre.pickle")
playlist_tag = pd.read_pickle("C:/Users/SAMSUNG/Desktop/KUBIG/2024 Fall/추천시스템/Mine/DATA/playlist.pickle")
id = pd.read_pickle("C:/Users/SAMSUNG/Desktop/KUBIG/2024 Fall/추천시스템/Mine/DATA/playlist_id.pkl")

model_path = "C:/Users/SAMSUNG/Desktop/KUBIG/2024 Fall/추천시스템/Model/cc.ko.300.vec"

In [3]:
# 전처리
playlist_tag.rename(columns = {'발매일':'발매년도', '좋아요':'좋아요 수'}, inplace = True)

chart_tag2 = chart_tag.drop('연도', axis = 1)
playlist_tag2 = playlist_tag.drop('플리', axis = 1)
songs_tag = pd.concat([chart_tag2, genre_tag, playlist_tag2])

In [6]:
# 합치기! 

songs_tag = songs_tag[songs_tag['발매년도'] != '-']
songs_tag.loc[:, '발매년도'] = songs_tag['발매년도'].apply(lambda x: str(x)[:4] if len(str(x)) > 4 else str(x))
songs_tag.loc[:, '좋아요 수'] = songs_tag['좋아요 수'].apply(lambda x: int(str(x).replace(',','')))
songs_tag = songs_tag.astype({'발매년도':'int32', '좋아요 수':'int32'})
songs_tag = songs_tag.drop_duplicates(['제목','가수'], keep = 'first')

In [7]:
# 태그 전처리 
hashtags = id.copy()

# 해시태그 없는 플레이리스트 제외
hashtags = hashtags[hashtags['tag'].str.contains("#")]
# 리스트로 전부 가져오기
only_tags = hashtags['tag'].tolist()
# "#" 제거하기
tags_fin  = []
for tag in only_tags:
  tags_fin.append(re.sub(r"[#\[\]\'\,]","",tag).split())
tags_fin[:2]

[['여름', '더워', '시원', '청량', '플리', '케이팝', 'kpop', '키오프'],
 ['팝송', '케이팝', '매장', '카페', '식당', '브금', '노래', '휴식', '신나는', '일']]

트랙 1) 일기 태그와 유사한 플리 찾아서 출력

트랙 2) 일기 태그와 가사 태그가 유사한 노래들

음악 취향 입력
일기 태그, 가사 태그 각각

#### 시도 1 - word2vec, OOV 삭제 

##### 트랙 1) 플리 태그 기반 플리 찾기

In [8]:
# 모델 학습 
word2vec_playlist = Word2Vec(sentences = tags_fin, vector_size = 50, window = 3, min_count = 1, sg = 1, workers = 4)

# 각 단어와 비슷한 단어 찾기... 성능 체크
print("이별이랑 비슷:", word2vec_playlist.wv.most_similar("이별"))
print("사랑이랑 비슷:", word2vec_playlist.wv.most_similar("사랑"))
print("여름이랑 비슷:", word2vec_playlist.wv.most_similar("여름"))

이별이랑 비슷: [('추억', 0.9958157539367676), ('우울', 0.9957407712936401), ('야경', 0.9955396056175232), ('잔잔한', 0.9953756332397461), ('새벽', 0.995330274105072), ('카페', 0.9953253865242004), ('인디', 0.9951081871986389), ('세련된', 0.9950740933418274), ('센치한', 0.9950241446495056), ('감성힙합', 0.9949735999107361)]
사랑이랑 비슷: [('세련된', 0.9960087537765503), ('발라드', 0.995893657207489), ('감성', 0.995736300945282), ('센치한', 0.9956501722335815), ('겨울', 0.9954304099082947), ('행복', 0.9953859448432922), ('편안한', 0.9953819513320923), ('우울', 0.9953790307044983), ('한강', 0.9953010678291321), ('노래방', 0.9952926635742188)]
여름이랑 비슷: [('발라드', 0.9968786239624023), ('청춘', 0.9961419105529785), ('겨울', 0.9959815740585327), ('피크닉', 0.9959700107574463), ('인디', 0.9958917498588562), ('Pop', 0.9958612322807312), ('케이팝', 0.9958339929580688), ('트렌디', 0.9958193302154541), ('설레는', 0.9956774711608887), ('세련된', 0.9956440925598145)]


###### 성능이 엄청 막 좋아보이지는 않...아

In [9]:
# 기존 태그 임베딩, 벡터 평균 함수 
def vector_and_average(input_lists, model):
  avg_vectors = []
  for list in input_lists:
    vectors = [model.wv[word] for word in list]
    avg = sum(vectors) / len(vectors)
    avg_vectors.append(avg)
  return avg_vectors

# 새로운 입력 - 모델에 있는 단어들만을 가지고 평균 벡터 생성
def new_tag_vector(tag, model):
  tag_vector = []
  for word in tag:
    if word in model.wv:
      tag_vector.append(model.wv[word])
    else:
      print(f"{word} 단어는 모델에 없습니다..")

  avg_vector = sum(tag_vector)/len(tag_vector)
  return avg_vector

# 코사인 유사도 함수
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

# 유사 해시태그 출력 함수
def similar_things(new_input, original_tags, model, num):
  new_vector = new_tag_vector(new_input, model)
  original_vectors = vector_and_average(original_tags, model)
  # 유사도 계산
  similarities = []
  for i in range(len(original_tags)):
    similarities.append(cos_sim(new_vector, original_vectors[i]))
  # 유사 해시태그 출력
  top_index = [idx for idx,_ in nlargest(num, enumerate(similarities), key = lambda x:x[1])] # enumerate 에서 두 번째 요소(유사도) 기준
  top_tags = [original_tags[idx] for idx in top_index] # 유사한 "태그"
  # print(top_index) # 유사한 "플리 index"
  return top_index, top_tags

In [10]:
# 기존 해시태그들 벡터화, 평균 계산
tags_vector = []
for tag in tags_fin:
  tags_vector.append(new_tag_vector(tag, word2vec_playlist))

print(f"첫 해시태그의 벡터화 : \n {tags_vector[0]}")

# 새로운 태그에 대한 평균 벡터 생성
new_tag = ['겨울', '사랑', '눈', '몽글몽글몽글몽글']
new_vector = new_tag_vector(new_tag, word2vec_playlist)

# 새로운 input과 유사한 애들
sim = similar_things(['신남', '댄스', '드라이브', '여름', '더워', '흥분'], tags_fin, word2vec_playlist, 5)
sim[0]

첫 해시태그의 벡터화 : 
 [ 0.04174288 -0.07175546  0.03454174  0.03228088 -0.03121519 -0.15712497
  0.15373649  0.20935765 -0.22676413 -0.11801913  0.02619883 -0.1937085
  0.15632737  0.0937292  -0.03358329  0.11670556  0.06203141  0.01118164
 -0.25351626 -0.10508794  0.01724208  0.14439414  0.24781309 -0.06193255
  0.15933226  0.00472437 -0.10877293  0.05716702 -0.16969669  0.0780718
  0.01862974 -0.07770737 -0.01380573 -0.06982583 -0.15681936  0.1373198
  0.12205312 -0.02337974  0.02634025 -0.01653233  0.19973798 -0.01456144
 -0.01904519 -0.00472036  0.31496435  0.01789968 -0.11823456 -0.0732379
  0.07841399  0.13612808]


In [13]:
# 추천 플레이리스트와 해시태그 
id.iloc[sim[0]]

,name,tag,id
301,output_17,"['#여름', '#신나는', '#팝', '#댄스', '#하우스', '#드라이브']",302
728,ozu_4,"['#인디', '#위로', '#힘들때', '#우울', '#지친일상', '#하루의끝']",729
161,moonsound_14,"['#여름', '#남자아이돌', '#댄스', '#드라이브', '#신나는', '#기분...",162
813,shorthair_18,"['#감성', '#추억', '#지하철', '#가사', '#보고싶다', '#사랑', ...",814
816,jjinmak_1,"['#응원', '#기분전환', '#위로', '#희망', '#힐링', '#휴식', '...",817


#### 트랙 2) 음악 취향 필터링, 가사 태그 기반 노래 찾기 

In [17]:
# 결측치 제거 
songs_tag_nona = songs_tag[~songs_tag['키워드'].isna()].copy()

# 키워드 + 감정 
songs_tag_nona['tags'] = songs_tag_nona['키워드'] + songs_tag_nona['감정']

songs_tags = songs_tag_nona['tags'].tolist()

In [18]:
# 모델 학습 
word2vec_songs = Word2Vec(sentences = songs_tags, vector_size = 100, window = 4, min_count = 1, workers = 4)

print("이별이랑 비슷:", word2vec_songs.wv.most_similar("이별"))
print("사랑이랑 비슷:", word2vec_songs.wv.most_similar("사랑"))
print("여름이랑 비슷:", word2vec_songs.wv.most_similar("여름"))

이별이랑 비슷: [('미련', 0.9972637891769409), ('눈물', 0.9962788820266724), ('허전함', 0.9951195120811462), ('아픔', 0.9931573271751404), ('헤어짐', 0.9928607940673828), ('후회', 0.9917554259300232), ('미안함', 0.9916582107543945), ('기억', 0.9906187057495117), ('비', 0.9873484969139099), ('미움', 0.9871535301208496)]
사랑이랑 비슷: [('우리', 0.9880443811416626), ('고백', 0.9850354194641113), ('영원', 0.9833611249923706), ('마음', 0.9817531704902649), ('기다림', 0.9812990427017212), ('당신', 0.9811688661575317), ('봄', 0.9762729406356812), ('시간', 0.976241409778595), ('포근함', 0.9761999845504761), ('약속', 0.9751516580581665)]
여름이랑 비슷: [('바다', 0.9967312812805176), ('하늘', 0.9942096471786499), ('크리스마스', 0.9890195727348328), ('감동', 0.9886176586151123), ('안락함', 0.9876447916030884), ('따뜻함', 0.9874204993247986), ('선물', 0.9872896671295166), ('안정', 0.987001359462738), ('기대', 0.9862778186798096), ('햇살', 0.9862753748893738)]


In [ ]:
# 기존 태그 벡터화 
original_vectors = vector_and_average(songs_tags, word2vec_songs)

print(len(original_vectors)) 

# 새로운 태그 벡터화 
idx, tag = similar_things(new_tag, songs_tags, word2vec_songs, 2)
songs_tag_nona.iloc[idx,:]

24088

#### 시도 2 - fasttext, OOV 반영

트랙 1 결과물 - filtered_hashtag 
- 모델에 없는 태그 제거 

##### 트랙 1) 플리 태그 기반 플리 찾기

In [10]:
# 사전학습 모델 로드 
fasttext = KeyedVectors.load_word2vec_format(
    model_path,
    binary=False,  # 텍스트 형식이므로 False
    encoding='utf-8',  # UTF-8 인코딩으로 로드
    unicode_errors='ignore'  # 디코딩 에러 무시
)

In [11]:
# 기존 tags_fin 중에 모델에 없는 거는 제거
def delete_missing_tags(tags_list):
    filtered_tags = []
    for tag in tags_list:
        filtered_list = [word for word in tag if word in fasttext]
        filtered_tags.append(filtered_list)
    return filtered_tags

In [12]:
# 기존 해시태그에서 모델에 없는 단어 제외
filtered_hashtag = delete_missing_tags(tags_fin)

# 혹시 전부 제거된 것이 있는지... -> 하나 있음...
n = 0
for tag in filtered_hashtag:
    if len(tag) == 0:
        n += 1
print(n)

1


In [13]:
# 모델에 있는 단어들만 남겨서 다시 플리 이름이랑 묶어 
id_filtered = hashtags.copy()
id_filtered['tag'] = filtered_hashtag

In [14]:
fasttext.most_similar("사랑한다", topn = 5)

C:\Users\SAMSUNG\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gensim\models\keyedvectors.py:849: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


[('웬수야', 0.6226855516433716),
 ('외쳐요', 0.5236533284187317),
 ('사랑하고', 0.5222238302230835),
 ('미안하다', 0.5081436634063721),
 ('사랑해줘', 0.5054067969322205)]

엥........ 모델 상태 왜이래,, 유사 단어 찾기 포기..... ㅠㅠ

In [15]:
# 남은 태그가 하나라면 제거
id_filtered = id_filtered[id_filtered['tag'].apply(lambda x: len(x) > 2)]
print(len(id_filtered))
# 1024 -> 994 로 줄어듦 
id_filtered.head(3)

994


,name,tag,id
0,bg_1,"[여름, 더워, 시원, 청량, 플리, 케이팝, kpop]",1
1,bg_2,"[팝송, 케이팝, 매장, 카페, 식당, 브금, 노래, 휴식, 신나는, 일]",2
2,bg_3,"[아침, 나른, 팝송, 플리, 노래, 산책, 침대, 잔잔, 휴식]",3


In [28]:
# 기존 태그 임베딩, 벡터 평균 함수 
def vector_and_average(input_lists):
    avg_vectors = []
    for list in input_lists:
        vectors = [fasttext[word] for word in list]
        avg = sum(vectors) / len(vectors)
        avg_vectors.append(avg)
    return avg_vectors 

# 새로운 태그 임베딩 함수 
def new_tag_vector(input):
    vectors = [fasttext[word] for word in input]
    avg_vectors = sum(vectors) / len(vectors)
    return avg_vectors

# 태그 유사도 계산
def cos_sim(A, B):
    return dot(A, B) / (norm(A) * norm(B))

# 가장 유사한 플리 출력 
def similar_things(input, original, num): # original : 비교대상 친구들 벡터화하기 전, 단어 형태 
    new_vector = new_tag_vector(input)
    original_vectors = vector_and_average(original)
    # 유사도 계산
    similarities = []
    for i in range(len(original)):
        similarities.append(cos_sim(new_vector, original_vectors[i]))
    # 유사 해시태그 
    top_index = [idx for idx,_ in nlargest(num, enumerate(similarities), key = lambda x:x[1])]
    top_tags = [original[idx] for idx in top_index]
    return top_index, top_tags 

In [17]:
# testing
new_tag = ['신남', '댄스', '드라이브', '여름', '더워', '흥분']

similar_things(new_tag, id_filtered['tag'])

([161, 657, 710, 159, 149],
 [['여름', '남자아이돌', '댄스', '드라이브', '신나는', '기분전환', '스트레스', '운동', '새벽', '바다'],
  ['퇴폐적인', '감각적인', '술', '밤', '저녁', '새벽', '라운지', '칵테일', 'bar', '기분전환'],
  ['가을', '감성', '플리', '기타', '인디', '발라드'],
  ['여름', '걸그룹', '댄스', '드라이브', '신나는', '기분전환', '스트레스', '운동', '취향저격', '추억'],
  ['여름', '스트레스', '걸그룹', '댄스', '기분전환', '여행', '신나는', '데이트', '자신감', '드라이브']])

##### 트랙 2) 음악 취향 필터링, 가사 태그 기반 노래 찾기

결과물 - filtered_songs_nona

In [86]:
# 키워드, 감정 없는 행 제거 (NaN 제거)
songs_tag_nona = songs_tag[~songs_tag['키워드'].isna()]

In [87]:
# 모델에 없는 단어 제외
keywords = songs_tag_nona.iloc[:,6].tolist()
feelings = songs_tag_nona.iloc[:,7].tolist()

filtered_keywords = delete_missing_tags(keywords)
filtered_feelings = delete_missing_tags(feelings)

In [88]:
# 전부 다 없어진 애들이 존재하나? 
n = 0
for tag in filtered_keywords:
    if len(tag) == 0:
        n += 1
print(n)

n = 0
for tag in filtered_feelings:
    if len(tag) == 0:
        n += 1
print(n)     

315
39


어어.... 싹 지워져버린 게 좀 있다만..... 어쩔 수 없지 뭐 일단 진행해야지...

In [89]:
filtered_songs = songs_tag_nona.copy()
filtered_songs['키워드'] = filtered_keywords
filtered_songs['감정'] = filtered_feelings

# 키워드, 감정 합쳐서 tags 정의
filtered_songs['tags'] = filtered_songs['키워드'] + filtered_songs['감정']

In [90]:
# 싹 지워져버린 친구들 제외
filtered_songs_nona = filtered_songs[filtered_songs.iloc[:,8].apply(lambda x: len(x) != 0)]
print(len(filtered_songs_nona))
filtered_songs_nona.head(3)

24083


,제목,가수,가사,발매년도,장르,좋아요 수,키워드,감정,tags
0,I Don't Think That I Like Her,Charlie Puth,Get her name and get her number Find out all o...,2022,POP,214990,"[그녀, 마음, 별]","[슬픔, 후회, 실망]","[그녀, 마음, 별, 슬픔, 후회, 실망]"
1,Dangerously,Charlie Puth,This is gonna hurt but I blame myself first ca...,2016,POP,217897,"[위험, 사랑, 진실]","[슬픔, 후회, 집착]","[위험, 사랑, 진실, 슬픔, 후회, 집착]"
2,STAY,"The Kid LAROI, Justin Bieber",I do the same thing I told you that I never wo...,2021,POP,293599,"[당신, 변화, 머무름]","[그리움, 후회, 불안]","[당신, 변화, 머무름, 그리움, 후회, 불안]"
